# Problem Set 3 

In [169]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy.optimize import minimize_scalar,minimize
import matplotlib.pyplot as plt
%matplotlib inline
from quantecon.markov import DiscreteDP
from scipy.stats import norm
from math import sqrt
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import brentq
import time

In [243]:
df = pd.read_csv('Pset3Data.csv')
#df.head()

Set initial values based on the data

In [259]:
np.random.seed(123456)
T = np.max(df.age)
N = np.max(df.id) # the number of individuals
nb = 100 # the number of draws for bootstrap. 

Bootstrap to calculate moments I need: average consumption and average assets.

In [24]:
c_mean = np.zeros((T, nb))
a_mean = np.zeros((T, nb))
for k in range(nb):
    draw = np.random.choice(N, N, replace = True)
    df_draw = df[df.id==draw[0]]
    for i in range(1,N):
        df_draw = df_draw.append(df[df.id==draw[i]])
    for t in range(T):
        age = t+1
        c_mean[t,k] = df_draw.cons[df_draw.age == age].mean()
        a_mean[t,k] = df_draw.asset[df_draw.age == age].mean()

In [81]:
bmoments = np.vstack([c_mean,a_mean])   # bmoments now is 20 by 100

Compute data moments

In [161]:
c_mean_data = np.zeros((T,1))
a_mean_data = np.zeros((T,1))
for t in range(T):
    age = t+1
    c_mean_data[t] = df.cons[df.age == age].mean()
    a_mean_data[t] = df.asset[df.age == age].mean()

In [162]:
data_moments = np.vstack([c_mean_data,a_mean_data]) # data_moments now is 20 by 1 , the first 10  moments is for c, and the remaining is for a.

In [420]:
data_moments[:,1]

array([0.72725394, 0.87605508, 1.05951965, 1.26605891, 1.4977864 ,
       1.73120871, 2.03193346, 2.42125249, 2.97024096, 3.93684768,
       0.        , 1.09787268, 2.05119868, 2.83550157, 3.45767702,
       3.90756111, 4.03047872, 3.88193591, 3.34537735, 2.2084611 ])

In [163]:
# repeat data moments for 100 times.
data_moments = np.repeat(data_moments, nb,axis=1)  # data_moments now is 20 by 100

In [85]:
dbmoments = np.subtract(bmoments,data_moments)
len(dbmoments)

20

In [104]:
variance = np.mean(dbmoments**2,axis = 1)    # variance is 1 by 20.
variance = np.delete(variance,10)

array([8.66526930e-05, 1.34420689e-04, 1.79918766e-04, 3.32098472e-04,
       5.08919227e-04, 6.38904344e-04, 7.12986523e-04, 1.01060539e-03,
       1.55858836e-03, 3.42917298e-03, 1.20696638e-03, 2.69554097e-03,
       3.70182411e-03, 5.56684392e-03, 6.65586732e-03, 6.20424822e-03,
       4.36237467e-03, 2.69453102e-03, 1.02120208e-03])

In [105]:
dataMomentsVCM = np.identity(2*T-1)

In [106]:
np.fill_diagonal(dataMomentsVCM, variance)     # dataMomentsVCM is a matrix of 20 by 20 

In [170]:
WeightMatrix = np.linalg.inv(dataMomentsVCM)

In [384]:
class CakeEating:
    def __init__(self, 
                 T = 10, 
                 r = 0.03,  
                 a0 = 0, 
                 minCons = 1e-5, 
                 ρ = 0.4, 
                 σ = 1,
                 N_inc = 3,       # number of income grid
                 N_grid = 3,   # number of asset grid
                 μ = 0):
        self.T, self.r, self.a0, self.minCons, self.ρ, self.σ, self.N_inc, self.N_grid, self.μ = T, r, a0, minCons, ρ, σ, N_inc, N_grid,μ    
        
    def Grid(self,y_max,y_min,N_grid):
        T,r,a0,minCons = self.T,self.r,self.a0,self.minCons
        
        gridMax = np.zeros(T)
        gridMax[0] = a0
        for num in range(T-1):
            gridMax[num+1]=(gridMax[num] + y_max)*(1+r)
        
        gridMin = np.zeros(T)
        gridMin[T-1] = -y_min + minCons
        for t in range(T-1):
            age = T-t-1
            gridMin[age-1] = gridMin[age]/(1+r) - y_min + minCons
        #for num in range(T):
         #   gridMin[num] = - y_min*(1/(1+r)-(1/(1+r))**(T-num+1))/(1-1/(1+r)) 
        
        loggrid = np.zeros((T,N_grid)) 
        grid = np.zeros((T,N_grid))
        for num in range(T):
            span = gridMax[num] - gridMin[num]
            loggrid[num,:] = np.linspace((np.log(1)),(np.log(1+span)),N_grid)
            grid[num,:] = np.exp(loggrid[num,:]) - 1 + gridMin[num]
        
        return(grid)
    
    def Utility(self,c,γ):
        return(c**(1-γ)/(1-γ))
    
    def MU(self,c,γ):
        return(c**(-γ))
    
    #generate a transition matrix
    def Pi(self):
        σ, ρ, N_inc, μ = self.σ,self.ρ,self.N_inc,self.μ
        sig_epsilon = σ/sqrt(1-ρ**2)    # SD of income
        
        #generate grid points epsilon
        epsilon = []
        epsilon.append(-3*sig_epsilon)
        n = list(range(2,N_inc+1))
        
        for num in n:
            epsilon.append(sig_epsilon*norm.ppf((num-1)/N_inc)+μ)

        epsilon.append(3*sig_epsilon)  #generate epsilon 
        
        #generate z_i
        z=[]
        for i in range(0,N_inc):
            z.append(N_inc*sig_epsilon*(norm.pdf((epsilon[i]-μ)/sig_epsilon)-norm.pdf((epsilon[i+1]-μ)/sig_epsilon)) + μ)
        
        #generate transition matrix
        Pi = np.zeros(shape=(N_inc,N_inc))
        for i in range(0,N_inc):
            for j in range(0,N_inc):
                Pi[i,j] = norm.cdf((epsilon[j+1]-μ*(1-ρ)-ρ*(epsilon[i+1]))/σ) - norm.cdf((epsilon[j]-μ*(1-ρ)-ρ*(epsilon[i+1]))/σ)
        
        sum = Pi.sum(axis=1)
        
        #normalize
        for i in range(0,N_inc):
            for j in range(0,N_inc):
                Pi[i,j] = Pi[i,j]/sum[i]  
        
        
        return Pi, z
    
    #generate states by the transition matrix, return to a vector of states for T periods  
    def State(self):
        Pi,T,N_inc = self.Pi, self.T,self.N_inc
        np.random.seed(123)
        state=np.zeros(T)
        state[0] = np.random.choice(range(N_inc), 1, replace = True)  # equal weight for each of income at the first period
        for t in range(1,T):
            p=[]
            for j in range(0,N_inc):
                p.append(Pi[int(state[t-1]),j])  #int is to convert the float to integer
                
            state[t] = np.random.choice(range(N_inc),1,replace=True,p=p)
        return state
    

In [385]:
model = CakeEating()
Pi,z = model.Pi()
y = np.exp(z)
T,r,N_inc,N_grid = model.T,model.r,model.N_inc,N_grid
a_grid = model.Grid(max(y),min(y),N_grid)
MU = model.MU

In [386]:
c_matrix = []
for j in range(N_grid):
    c_matrix.append(a_grid[T-1]+y[j]) 
c = np.zeros((T,N_grid,N_inc))
for j in range(N_inc):
    for i in range(N_grid):
        c[T-1,i,j] = c_matrix[j][i] 

In [387]:
def Euler_linear(a0,index,a1,β,γ,f):
    ConsumptionToday = a0 + y[index] - a1/(1+r)
    w = Pi[index]
    data = np.zeros(N_inc)
    for i in range(N_inc):
        data[i] = (f(y[i],a1))**(-γ)
    #data = np.concatenate((MU2(y[0],a1),MU2(y[1],a1),MU2(y[2],a1),MU2(y[3],a1),MU2(y[4],a1)),axis = None)
    ExpectedMU = np.average(data, weights = w)
    Euler = MU(ConsumptionToday,γ) - (1+r)*β*ExpectedMU
    return Euler

In [388]:
def Euler_linear_cons(β,γ):
    c_Euler_linear = np.zeros((T,N_grid,N_inc))
    c_Euler_linear[T-1] = c[T-1]
    optimal_a_linear = np.zeros((T,N_grid,N_inc))
    for t in range(T-1):
        lb = np.min(a_grid[T-t-1])
        yy, aa = np.meshgrid(y, a_grid[T-t-1])
        MU2_linear = interp2d(y,a_grid[T-t-1],(MU(c_Euler_linear[T-t-1],γ))**(-1/γ))
        for i in range(N_grid):
            for j in range(N_inc):
                ub = (1+r)*(a_grid[T-t-2,i]+y[j]-1e-5) 
                diff = ub - lb
                if diff < 1e-5 or Euler_linear(a_grid[T-t-2,i],j,lb,β,γ,MU2_linear) > 0:  # if constrained
                    optimal_a_linear[T-t-1,i,j] = lb
                    c_Euler_linear[T-t-2,i,j] = a_grid[T-t-2,i] + y[j] - optimal_a_linear[T-t-1,i,j]/(1+r)
                else:
                    optimal_a_linear[T-t-1,i,j] = optimize.brentq(lambda a1: Euler_linear(a_grid[T-t-2,i],j,a1,β,γ,MU2_linear),lb,ub)
                    c_Euler_linear[T-t-2,i,j] = a_grid[T-t-2,i] + y[j] - optimal_a_linear[T-t-1,i,j]/(1+r)
    return(c_Euler_linear)

In [423]:
def findMomentVec(β,γ):
    
    c_Euler_linear = Euler_linear_cons(β,γ)
    
    np.random.seed(20200202)
    num_ind = 100
    state = np.zeros((num_ind,T))

    for i in range(num_ind):
        state[i,0] = np.random.choice(range(N_inc), 1, replace = True)
        
        for t in range(1,T):
            p=[]
            for j in range(N_inc):
                p.append(Pi[int(state[i,t-1]),j])  #int is to convert the float to integer
            
            state[i,t] = np.random.choice(range(N_inc),1,replace=True,p=p)
    
    cons = np.zeros([num_ind,T])
    a1 = np.zeros([num_ind,T])
    y_t = np.zeros([num_ind,T])
    a0 = model.a0

    for t in range(T):
        a = a_grid[t]
        yy, aa = np.meshgrid(y,a)
        consumption = interp2d(y,a,c_Euler_linear[t])   
        for i in range(num_ind):
            index = int(state[i,t])
            y_t[i,t] = y[index]
            if t == 0:
                cons[i,t] = consumption(y_t[i,t],a0)
                a1[i,t] = (a0 + y_t[i,t] - cons[i,t])*(1+r)
            else:
                cons[i,t] = consumption(y_t[i,t],a1[i,t-1])
                a1[i,t] = (a1[i,t-1] + y_t[i,t]- cons[i,t])*(1+r)
    
    c_mean = np.zeros(T)
    a_mean = np.zeros(T)
    for t in range(T):
        c_mean[t] = np.average(cons[:,t])
        a_mean[t] = np.average(a1[:,t])
        
    gamma_hat = np.concatenate((c_mean, a_mean[1:]), axis=None)
    data_moments_firstColu = data_moments[:,0]
    data_moments_delete = np.delete(data_moments_firstColu,10)
    diff = gamma_hat - data_moments_delete
    return(diff)

In [297]:
# trial 
def findMomentVec(β,γ):
    
    c_Euler_linear = Euler_linear_cons(β,γ)
    
    np.random.seed(20200202)
    num_ind = N
    id_draw = np.random.choice(N,N,replace=True)
    print(id_draw)
    
    cons = np.zeros([num_ind,T])
    a1 = np.zeros([num_ind,T])
    a0 = model.a0

    for t in range(T):
        a = a_grid[t]
        yy, aa = np.meshgrid(y,a)
        consumption = interp2d(y,a,c_Euler_linear[t]) 
        age = t+1
        for i in range(num_ind):
            income = df.inc[(df.age == age) & (df.id==id_draw[i])]
            asset = df.asset[(df.age == age)&(df.id == id_draw[i])]
            if t == 0:
                cons[i,t] = consumption(income, asset)
                a1[i,t] = (a0 + income - cons[i,t])*(1+r)
            else:
                cons[i,t] = consumption(income,asset)
                a1[i,t] = (a1[i,t-1] + income- cons[i,t])*(1+r)
    
    c_mean = np.zeros(T)
    a_mean = np.zeros(T)
    for t in range(T):
        c_mean[t] = np.average(cons[:,t])
        a_mean[t] = np.average(a1[:,t])
        
    gamma_hat = np.concatenate((c_mean, a_mean[1:]), axis=None)
    data_moments_firstColu = data_moments[:,0]
    data_moments_delete = np.delete(data_moments_firstColu,10)
    diff = gamma_hat - data_moments_delete
    return(diff)

In [424]:
def getCriterionFunction(θ):
    β,γ = θ[0], θ[1]
    diff = findMomentVec(θ[0], θ[1])
    criterionFunction = diff@WeightMatrix@diff.T
    return (criterionFunction)

In [440]:
θ0 = np.array([0.9,1.5])  #set initial value
bnd = ((0.5,1), (1,10))    # set bound for beta and gamma
oout = minimize(getCriterionFunction,θ0,bounds=bnd)

In [441]:
oout

      fun: 7936.250027033526
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-210.80759325,  -38.98294381])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 8
   status: 0
  success: True
        x: array([0.66886837, 9.08680667])

In [ ]:
# initial value 0.97, 1.8 ; range (0.5,1) (1.1,10); N_ind = 3000